# OBS!! Kör inte detta lokalt utan i peach lab

In [ ]:
# Kör inte detta lokalt utan i notebook

# Installera implicit
pip install implicit

# Starta om din kernel om du blir uppmanad till det (menyn kernel -> restart kernel...) 

In [ ]:
# Ladda upp history.json till peach lab.
# Lägg den i samma folder som denna notebook.

In [ ]:
import json
file = open('history.json')
stuff = file.read()
file.close()
history = json.loads(stuff)

In [ ]:
# Titta på de första 10 elementen
# History innehåller avsnittsinteraktioner från användare de senaste 60 dagarna.
# Det är ett subset av alla användare (eftersom det skulle vara väldigt mycket data annars)
# En interaktion är en spelning av ett avsnitt med ett särskilt id.
history[:10]

In [ ]:
# Nu ska vi stoppa in det i implicit-libraryt
# Ni får gärna försöka förstå vad som händer här men det är mest kod för att få in datat på rätt sätt in i collaborative filtering-algoritmen

import json
import implicit
import numpy as np
from scipy.sparse import csr_matrix

from pipe_algorithms_lib.utils import log, delta

FACTORS = 10
ITERATIONS = 5


def _remove_dups_in_list(lst):
    return list(dict.fromkeys(lst))


def _remove_false_none_in_list(lst):
    return [x for x in lst if x]


def _extract_values_from_tuples(list_of_tuples, index):
    return list( i[index] for i in list_of_tuples)


def _load_data():
    file = open('history.json')
    stuff = file.read()
    file.close()
    history = json.loads(stuff)

    interactions = []
    for (user_id, hist) in history:
        for media_id in hist:
            interactions.append((user_id, media_id))

    return interactions


def _convert_data_to_sparse_matrix(users, episodes):
    """
        Create arrays to feed into the sparse matrix creation
        We create a static rank of 1 for all episodes a user has watched
    """
    watched_rating = 1
    rating_array = np.full((len(users)), watched_rating)

    users_array = np.asarray(users)

    episodes_array = np.asarray(episodes)

    return csr_matrix((rating_array, (users_array, episodes_array)))


def create_model():
    """
        Create mappings for users and episodes
            sparse matrix only want numbers to calculate
            so we need to convert userID and EpisodeID to an int and then keep track of it.
            We use dictionaries to do this dict() so we then 
            can do a very quick look up through Redis
            ex. {0: '4230218b-dbef-3ee2-b7d2-f0bb568b2b2b'} --> 
        Create the sparse matrix which is used to train the recommendation model
        Train the reco model
        Save the result to redis to be fetched from an endpoint
    """
    data = _load_data()
    data = list(set(data))
    log(f'Number of events loaded: {len(data)}')

    only_users = _extract_values_from_tuples(data, 0)
    filtered_only_users = _remove_false_none_in_list(only_users)
    unique_only_users = _remove_dups_in_list(only_users)
    dict_of_unique_users = dict(enumerate(unique_only_users))
    inv_users_mapping = {v: k for k, v in dict_of_unique_users.items()}
    users_as_ints = [inv_users_mapping[x] for x in filtered_only_users]

    only_episodes = _extract_values_from_tuples(data, 1)
    filtered_only_episodes = _remove_false_none_in_list(only_episodes)
    unique_only_episodes = _remove_dups_in_list(only_episodes)
    dict_of_unique_episodes = dict(enumerate(unique_only_episodes))    
    inv_episodes_mapping = {v: k for k, v in dict_of_unique_episodes.items()}
    episodes_as_ints = [inv_episodes_mapping[x] for x in filtered_only_episodes]

    sparse_matrix = _convert_data_to_sparse_matrix(users_as_ints, episodes_as_ints)
    log(f'Matrix shape: {sparse_matrix.shape}')

    # Skapa modellen för collaborative filtering!
    # Implicit-libraryt heter så för att det är implicita features (lyssning är implicit gillande)
    # AlternatingLeastSquares är en av flera solvers till matrixfaktoriseringsproblemet
    model = implicit.als.AlternatingLeastSquares(factors=FACTORS, iterations=ITERATIONS)
    # train the model
    model.fit(sparse_matrix)

    return model, inv_users_mapping, dict_of_unique_episodes, sparse_matrix

In [ ]:
# Kör modellskapar-funktionen (som också spottar ut lite andra vettiga saker)
model, inv_users_mapping, dict_of_unique_episodes, sparse_matrix = create_model()

In [ ]:
# Funktionen för att rekommendera till användare
def recommend_items_to_user(model, inv_users_mapping, dict_of_unique_episodes, sparse_matrix, user_id, size):
    user_model_id = inv_users_mapping.get(user_id)
    if user_model_id is None:
        return []

    recs = model.recommend(user_model_id, sparse_matrix[user_model_id], N=size)
    recs = zip(*recs)
    rec_result = [(dict_of_unique_episodes[rec[0]], float(rec[1])) for rec in recs if dict_of_unique_episodes[rec[0]]]

    return rec_result

In [ ]:
# Rekommendera till en användare!
recommend_items_to_user(model, inv_users_mapping, dict_of_unique_episodes, sparse_matrix, '2bec17de-86ae-4625-a040-752ee0871d17', 10)

# Egen uppgift
- Testa resultaten. Ser det rimligt ut?
  - använd pythons requests-library för att kolla vad idna användaren lyssnat på innebär från SRs api https://api.sr.se/api/documentation/v2/index.html
- Gör tutorials i Peach!
  - kolla https://git.ebu.io/pipe/lab-tutorials
  - kolla pipe_algorithms -> notebooks -> tutorials